In [1]:
from astropy.io import ascii
from astropy.io import fits
from astropy.table import setdiff, Table, unique, vstack
from matplotlib.patches import Ellipse
import aplpy
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.spatial as spatial

In [ ]:
def parallax_plot(cat_1, cat_1_ra, cat_1_dec, cat_2, cat_2_ra, cat_2_dec, parallax, arc_range, k, plot_dance = False):
    
    %run Crossmatch.ipynb
    %run ds9_region.ipynb
    
    x_axis = parallax
    y_axis = "Number Counts"
    
    # Cross-matches the filter catalog with the first catalog
    cross_filter_cat = cross_match(cat_1, cat_1_ra, cat_1_dec, cat_2, cat_2_ra, cat_2_dec, arc_range)
    
    cross_filter_cat = reset_main_catalog_columns(cat_1, cross_filter_cat)
    
    cross_filter_cat[parallax].fill_value = - 999
    
    cross_filter_cat = cross_filter_cat.filled()
    
    cross_filter_cat.remove_rows([cross_filter_cat[parallax] == - 999])
    
    arr_x_total = np.array(cross_filter_cat[parallax])
    
    #arr_x_total[arr_x_total == - 999] = np.nan
    #arr_x_total = arr_x_total[np.isfinite(arr_x_total)]
    
    arr_x = np.array(cross_filter_cat[parallax])
    #arr_x[arr_x == - 999] = np.nan
    #arr_x = arr_x[np.isfinite(arr_x)]
    mean_arr_x = np.nanmean(arr_x)
    std_arr_x = np.absolute(np.nanstd(arr_x))
    
    # Reduces the values which will be used to calculate the linear-fit to an interval around the mean
    arr_x[arr_x < (mean_arr_x - k * std_arr_x)] = np.nan
    arr_x[arr_x > (mean_arr_x + k * std_arr_x)] = np.nan
    
    # First removes the entries from ARR_Y which correspond to NaN values in ARR_X
    # The other way around, ARR_X would have less rows than ARR_Y and an error would occur
    arr_x = arr_x[np.isfinite(arr_x)]
    
    arr_x = np.absolute(arr_x)
    print(np.nanmedian(arr_x))
    distance_parallax = np.absolute(1000 / arr_x)
    
    distance_parallax = distance_parallax[np.isfinite(distance_parallax)]
    
    plt.figure(1)
    if plot_dance == True:
        dance = ascii.read('/home/yobd/Documents/CatalogsCrA/New_Run/DANCE.dat')
        dance_parallax = np.array(dance['plx'])
        n_2, bins_2, patches_2 = plt.hist(dance_parallax, bins = 16, histtype = 'barstacked', color = 'blue')
    n, bins, patches = plt.hist(arr_x, bins = 16, histtype = 'barstacked', color = 'green')
    plt.xlabel(x_axis + " (mas)", fontsize = 'xx-large')
    plt.ylabel(y_axis, fontsize = 'xx-large')
    plt.savefig(fname = '/home/yobd/Documents/Parallax/parallax_histogram.png')
    plt.show()
    
    col = Table()
    
    col[parallax] = arr_x
    
    rejects = setdiff(cross_filter_cat, col, keys = [parallax])
    
    if len(rejects) != 0:
        
        rejects = unique(rejects, keys = [parallax], keep = 'first')
        
        rejects = reset_main_catalog_columns(cat_1, rejects)
        
        ascii.write(rejects, '/home/yobd/Documents/Parallax/rejects.cat', format = 'csv', overwrite = True)
        
        create_ds9_region(rejects['ra'], rejects['dec'], '/home/yobd/Documents/Parallax/rejects.reg')
        
        accepted = setdiff(cross_filter_cat, rejects, keys = [parallax])
    
    else:
        
        accepted = Table.copy(cross_filter_cat)
    
    accepted = unique(accepted, keys = [parallax], keep = 'first')
    
    accepted = reset_main_catalog_columns(cat_1, accepted)
    
    ascii.write(accepted, '/home/yobd/Documents/Parallax/accepted.cat', format = 'csv', overwrite = True)
    
    create_ds9_region(accepted['ra'], accepted['dec'], '/home/yobd/Documents/Parallax/accepted.reg')
    
    plt.figure(2)
    plt.scatter(accepted[parallax], accepted['I_MAG_SUBARU'], marker = '.')
    plt.xlabel("Parallax (mas)", fontsize = 'xx-large')
    plt.ylabel('I (mag)', fontsize = 'xx-large')
    plt.show()

def parallax_selec(cat, parallax, arc_range, k, plot_dance = False):
    
    %run Crossmatch.ipynb
    %run ds9_region.ipynb
    
    x_axis = "Parallax"
    y_axis = "Number Counts"
    
    cat[parallax].fill_value = - 999
    
    cat = cat.filled()
    
    cat.remove_rows([cat[parallax] == - 999])
    
    temp_cat = Table.copy(cat)
    
    temp_cat.remove_rows([temp_cat[parallax] <= 2])
    
    arr_x_total = np.array(temp_cat[parallax])
    
    #arr_x_total[arr_x_total == - 999] = np.nan
    #arr_x_total = arr_x_total[np.isfinite(arr_x_total)]
    
    arr_x = np.array(temp_cat[parallax])
    #arr_x[arr_x == - 999] = np.nan
    #arr_x = arr_x[np.isfinite(arr_x)]
    mean_arr_x = np.nanmean(arr_x)
    std_arr_x = np.absolute(np.nanstd(arr_x))
    
    # Reduces the values which will be used to calculate the linear-fit to an interval around the mean
    arr_x[arr_x < (mean_arr_x - k * std_arr_x)] = np.nan
    arr_x[arr_x > (mean_arr_x + k * std_arr_x)] = np.nan
    
    # First removes the entries from ARR_Y which correspond to NaN values in ARR_X
    # The other way around, ARR_X would have less rows than ARR_Y and an error would occur
    arr_x = arr_x[np.isfinite(arr_x)]
    
    arr_x = np.absolute(arr_x)
    
    print("Mean:", mean_arr_x)
    
    print("Standard deviation:", std_arr_x)
    
    distance_parallax = np.absolute(1000 / arr_x)
    distance_parallax = distance_parallax[np.isfinite(distance_parallax)]
    
    col = Table()
    
    col[parallax] = arr_x
    
    rejects = setdiff(cat, col, keys = [parallax])
    
    if len(rejects) != 0:
        
        rejects = unique(rejects, keys = [parallax], keep = 'first')
        
        rejects = reset_main_catalog_columns(cat, rejects)
        
        ascii.write(rejects, '/home/yobd/Documents/Parallax/rejects.cat', format = 'csv', overwrite = True)
        
        create_ds9_region(rejects['ra'], rejects['dec'], '/home/yobd/Documents/Parallax/rejects.reg')
        
        accepted = setdiff(cat, rejects, keys = [parallax])
    
    else:
        
        accepted = Table.copy(temp_cat)
    
    accepted = unique(accepted, keys = [parallax], keep = 'first')
    
    accepted = reset_main_catalog_columns(cat, accepted)
    
    ascii.write(accepted, '/home/yobd/Documents/Parallax/accepted.cat', format = 'csv', overwrite = True)
    
    create_ds9_region(accepted['ra'], accepted['dec'], '/home/yobd/Documents/Parallax/accepted.reg')
    
    plt.figure(1)
    if plot_dance == True:
        dance = ascii.read('/home/yobd/Documents/CatalogsCrA/New_Run/DANCE.dat')
        dance_parallax = np.array(dance['plx'])
        n_2, bins_2, patches_2 = plt.hist(dance_parallax, bins = 16, histtype = 'barstacked', color = 'blue', label = 'DANCE Sources')
    n, bins, patches = plt.hist(arr_x, bins = 16, histtype = 'barstacked', color = 'green', label = 'Accepted')
    n_rej, bins_rej, patches_rej = plt.hist(rejects[parallax], bins = 16, histtype = 'barstacked', color = 'red', label = 'Rejected')
    plt.xlabel(x_axis + " (mas)", fontsize = 'xx-large')
    plt.ylabel(y_axis, fontsize = 'xx-large')
    plt.legend(fontsize = 'medium', loc = 1)
    plt.show()
    
    plt.figure(2)
    plt.scatter(rejects[parallax], rejects['I_MAG_SUBARU'], marker = '.', color = 'red', label = 'Rejected')
    plt.scatter(accepted[parallax], accepted['I_MAG_SUBARU'], marker = '.', color = 'green', label = 'Accepted')
    plt.xlabel(x_axis + " (mas)", fontsize = 'xx-large')
    plt.ylabel('I (mag)', fontsize = 'xx-large')
    plt.legend(fontsize = 'medium', loc = 4)
    plt.show()
    
    return accepted

def proper_motions_plot(cat_1_path, cat_1_ra, cat_1_dec, cat_2_path, cat_2_ra, cat_2_dec, pm_ra, pm_dec, arc_range, k):
    
    %run Crossmatch.ipynb
    %run ds9_region.ipynb
    
    x_axis = pm_ra
    y_axis = pm_dec
    
    # Reads the catalog file into a Table object
    cat_1 = ascii.read(cat_1_path)
    
    # Reads the catalog file into a Table object
    cat_2 = ascii.read(cat_2_path)
    
    # Cross-matches the filter catalog with the first catalog
    cross_filter_cat = cross_match(cat_1, cat_1_ra, cat_1_dec, cat_2, cat_2_ra, cat_2_dec, arc_range)
    
    cross_filter_cat[pm_ra].fill_value = -999
    cross_filter_cat[pm_dec].fill_value = -999
    
    cross_filter_cat = cross_filter_cat.filled()
    
    arr_x_total = np.array(cross_filter_cat[pm_ra])
    
    arr_y_total = np.array(cross_filter_cat[pm_dec])
    
    arr_x_total[arr_x_total == - 999] = np.nan
    arr_y_total = arr_y_total[np.isfinite(arr_x_total)]
    arr_x_total = arr_x_total[np.isfinite(arr_x_total)]
    arr_y_total[arr_y_total == - 999] = np.nan
    arr_x_total = arr_x_total[np.isfinite(arr_y_total)]
    arr_y_total = arr_y_total[np.isfinite(arr_y_total)]
    
    arr_x = np.array(cross_filter_cat[pm_ra])
    arr_x[arr_x == - 999] = np.nan
    mean_arr_x = np.nanmean(arr_x)
    std_arr_x = np.absolute(np.nanstd(arr_x))
    
    arr_y = np.array(cross_filter_cat[pm_dec])
    arr_y[arr_y == - 999] = np.nan
    mean_arr_y = np.nanmean(arr_y)
    std_arr_y = np.absolute(np.nanstd(arr_y))
    
    # Reduces the values which will be used to calculate the linear-fit to an interval around the mean
    arr_y[arr_y < (mean_arr_y - k * std_arr_y)] = np.nan
    arr_y[arr_y > (mean_arr_y + k * std_arr_y)] = np.nan
    
    # First removes the entries from ARR_X which correspond to NaN values in ARR_Y
    # The other way around, ARR_Y would have less rows than ARR_X and an error would occur
    arr_x = arr_x[np.isfinite(arr_y)]
    arr_y = arr_y[np.isfinite(arr_y)]
    
    # Reduces the values which will be used to calculate the linear-fit to an interval around the mean
    arr_x[arr_x < (mean_arr_x - k * std_arr_x)] = np.nan
    arr_x[arr_x > (mean_arr_x + k * std_arr_x)] = np.nan
    
    # First removes the entries from ARR_Y which correspond to NaN values in ARR_X
    # The other way around, ARR_X would have less rows than ARR_Y and an error would occur
    arr_y = arr_y[np.isfinite(arr_x)]
    arr_x = arr_x[np.isfinite(arr_x)]
    
    col = Table()
    
    col[pm_ra] = arr_x
    col[pm_dec] = arr_y
    
    rejects = setdiff(cross_filter_cat, col, keys = [pm_ra, pm_dec])
    
    rejects = unique(rejects, keys = [pm_ra, pm_dec], keep = 'first')
    
    plt.figure(1)
    plt.scatter(arr_x, arr_y, s = 1, marker = ',')
    plt.xlabel(r'$\mu_{\alpha}$ (mas/yr)', fontsize = 'xx-large')
    plt.ylabel(r'$\mu_{\delta}$ (mas/yr)', fontsize = 'xx-large')
    plt.axhline(y = 0, color = 'k', linestyle = '--', linewidth = 0.7)
    plt.axvline(x = 0, color = 'k', linestyle = '--', linewidth = 0.7)
    #plt.xlabel("\u03BC_\u03B4 (mas/yr)", fontsize = 'xx-large')
    #plt.ylabel("\u03BC_\u03B4 (mas/yr)", fontsize = 'xx-large')
    plt.savefig(fname = '/home/yobd/Documents/ProperMotions/accepted.png')
    
    plt.show()
    
    plt.figure(2)
    plt.scatter(arr_x_total, arr_y_total, s = 1, marker = ',')
    plt.scatter(arr_x, arr_y, s = 1, marker = ',')
    plt.xlabel(r'$\mu_{\alpha}$' + " (mas/yr)", fontsize = 'xx-large')
    plt.ylabel(r'$\mu_{\delta}$' + " (mas/yr)", fontsize = 'xx-large')
    plt.axhline(y = 0, color = 'k', linestyle = '--', linewidth = 0.7)
    plt.axvline(x = 0, color = 'k', linestyle = '--', linewidth = 0.7)
    #plt.xlabel("\u03BC_\u03B4 (mas/yr)", fontsize = 'xx-large')
    #plt.ylabel("\u03BC_\u03B4 (mas/yr)", fontsize = 'xx-large')
    plt.savefig(fname = '/home/yobd/Documents/ProperMotions/accepted_with_rejects.png')
    
    plt.show()
    
    rejects = reset_main_catalog_columns(cat_1, rejects)
    
    ascii.write(rejects, '/home/yobd/Documents/ProperMotions/rejects.dat', format = 'csv', overwrite = True)
    
    cat = ascii.read('/home/yobd/Documents/ProperMotions/rejects.dat')
    
    create_ds9_region(cat['ra'], cat['dec'], '/home/yobd/Documents/ProperMotions/rejects.reg')
    
    accepted = setdiff(cross_filter_cat, rejects, keys = [pm_ra, pm_dec])
    
    accepted = unique(accepted, keys = [pm_ra, pm_dec], keep = 'first')
    
    accepted = reset_main_catalog_columns(cat_1, accepted)
    
    ascii.write(accepted, '/home/yobd/Documents/ProperMotions/accepted.dat', format = 'csv', overwrite = True)
    
    cat = ascii.read('/home/yobd/Documents/ProperMotions/accepted.dat')
    
    create_ds9_region(cat['ra'], cat['dec'], '/home/yobd/Documents/ProperMotions/accepted.reg')

def proper_motions_selec(confirmed, radius, list_of_points, pmra_name = 'pmRA', pmdec_name = 'pmDE', pmra_error_name = 'e_pmRA', pmdec_error_name = 'e_pmDE'):
    
    confirmed[pmra_name] = confirmed[pmra_name].filled(999)
    
    confirmed.remove_rows([confirmed[pmra_name] == 999])
    
    confirmed.remove_rows([confirmed['RUWE'] > 1.4])
    
    pm_ra = np.nanmean(confirmed[pmra_name])
    
    pm_dec = np.nanmean(confirmed[pmdec_name])
    
    list_of_points_ra = np.array(list_of_points[pmra_name])
    
    list_of_points_dec = np.array(list_of_points[pmdec_name])
    
    points = np.vstack((list_of_points_ra, list_of_points_dec)).T
    
    point_tree = spatial.cKDTree(points)
    
    pmra_median = np.nanmedian(confirmed[pmra_name])
    pmra_std = np.nanstd(confirmed[pmra_name])
    pmdec_median = np.nanmedian(confirmed[pmdec_name])
    pmdec_std = np.nanstd(confirmed[pmdec_name])
    
    confirmed.remove_rows([confirmed[pmra_name] < pmra_median - 3 * pmra_std])
    confirmed.remove_rows([confirmed[pmra_name] > pmra_median + 3 * pmra_std])
    
    confirmed.remove_rows([confirmed[pmdec_name] < pmdec_median - 3 * pmdec_std])
    confirmed.remove_rows([confirmed[pmdec_name] > pmdec_median + 3 * pmdec_std])
    
    print(np.nanmean(confirmed[pmra_name]))
    pmra_mean = np.nanmean(confirmed[pmra_name])
    pmra_std = np.nanstd(confirmed[pmra_name])
    pmdec_mean = np.nanmean(confirmed[pmdec_name])
    pmdec_std = np.nanstd(confirmed[pmdec_name])
    
    #print(pmra_median, pmra_std)
    #print(pmdec_median, pmdec_std)
    
    #confirmed = confirmed[np.where(np.logical_and(np.greater_equal(confirmed['pmra'], pmra_median - 2 * pmra_std), np.less_equal(confirmed['pmra'], pmra_median + 2 * pmra_std)))]
    
    #confirmed = confirmed[(confirmed['pmra'] > pmra_mean - 2 * pmra_std) & (confirmed['pmra'] < pmra_mean + 2 * pmra_std) & (confirmed['pmdec'] > pmdec_mean - 2 * pmdec_std) & (confirmed['pmdec'] < pmdec_mean + 2 * pmdec_std)]
    #confirmed = confirmed[confirmed['pmra'] < pmra_median + 2 * pmra_std]
    
    #confirmed = confirmed[confirmed['pmdec'] > pmdec_median - 2 * pmdec_std]
    #confirmed = confirmed[confirmed['pmdec'] < pmdec_median + 2 * pmdec_std]
        
    #circle = plt.Circle((pm_ra, pm_dec), 5, fill = False, color = 'k')
    
    accepted_points = list_of_points[point_tree.query_ball_point([pmra_median, pmdec_median], radius)]
    
    pmra_mean_err = np.nanmean(confirmed[pmra_error_name])
    
    pmdec_mean_err = np.nanmean(confirmed[pmdec_error_name])
    
    #print(confirmed['pmra_error'], confirmed['pmdec_error'])
    #print(pmra_mean_err, pmdec_mean_err)
    print("Mean proper motions:", pmra_mean, "+/-", pmra_std, pmdec_mean, "+/-", pmdec_std)
    print("Ellipse center:", pmra_median, "+/-", pmra_std, pmdec_median, "+/-", pmdec_std)
    ellipse = Ellipse((pmra_median, pmdec_median), 11, 11, fill = False, color = 'k')
    
    #in_ellipse = np.empty((len(list_of_points), 2), dtype = np.float64)
    
    in_ellipse_ra = []
    in_ellipse_dec = []
    #print(in_ellipse)
    i = 0
    #print(np.shape(in_ellipse))
    while i < len(list_of_points):
        
        if ellipse.contains_point((list_of_points_ra[i], list_of_points_dec[i]), radius = 0):
            #print(list_of_points_ra[i], list_of_points_dec[i])
            #np.append(in_ellipse, [[list_of_points_ra[i], list_of_points_dec[i]]], axis = 0)
            in_ellipse_ra.append(list_of_points_ra[i])
            in_ellipse_dec.append(list_of_points_dec[i])
            #in_ellipse[i] = [list_of_points_ra[i], list_of_points_dec[i]]
            #print(in_ellipse[i])
            
            i = i + 1
        
        else:
            i = i + 1
    
    #in_ellipse = np.array(in_ellipse)
    #print(in_ellipse[0])
    
    confirmed_in_ellipse_ra = []
    confirmed_in_ellipse_dec = []
    #print(in_ellipse)
    j = 0
    #print(np.shape(in_ellipse))
    while j < len(confirmed):
        
        if ellipse.contains_point((confirmed[j][pmra_name], confirmed[j][pmdec_name]), radius = 0):
            #print(list_of_points_ra[i], list_of_points_dec[i])
            #np.append(in_ellipse, [[list_of_points_ra[i], list_of_points_dec[i]]], axis = 0)
            confirmed_in_ellipse_ra.append(confirmed[j][pmra_name])
            confirmed_in_ellipse_dec.append(confirmed[j][pmdec_name])
            #in_ellipse[i] = [list_of_points_ra[i], list_of_points_dec[i]]
            #print(in_ellipse[i])
            
            j = j + 1
        
        else:
            j = j + 1
    
    plt.rcdefaults()
    plt.rc('xtick',labelsize = 20)
    plt.rc('ytick',labelsize = 20)
    
    fig, ax = plt.subplots(figsize = (10, 10))
    ax.set_aspect('equal')
    plt.scatter(list_of_points_ra, list_of_points_dec, s = 60, marker = '.', label = 'CMD-Selected Sources')
    #plt.scatter(accepted_points['pmra'], accepted_points['pmdec'], marker = 'D')
    #plt.scatter(confirmed['pmra'], confirmed['pmdec'], marker = '.')
    plt.scatter(confirmed[pmra_name], confirmed[pmdec_name], s = 90, marker = '*', label = 'CrA Members', color = 'g')
    plt.scatter(in_ellipse_ra, in_ellipse_dec, s = 90, marker = '.', label = 'CMD+PM-Selected')
    plt.xlabel(r'$\mu_{\alpha}$' + " [mas / yr]", fontsize = 30)
    plt.ylabel(r'$\mu_{\delta}$' + " [mas / yr]", fontsize = 30)
    plt.axhline(y = 0, color = 'k', linestyle = '--', linewidth = 0.7)
    plt.axvline(x = 0, color = 'k', linestyle = '--', linewidth = 0.7)
    #plt.hlines(pm_dec - radius, pm_ra - radius, pm_ra + radius)
    #plt.hlines(pm_dec + radius, pm_ra - radius, pm_ra + radius)
    #plt.vlines(pm_ra - radius, pm_dec - radius, pm_dec + radius)
    #plt.vlines(pm_ra + radius, pm_dec - radius, pm_dec + radius)
    x_lim = plt.xlim(-20, 20)
    y_lim = plt.ylim(-35, 20)
    plt.gca().add_artist(ellipse)
    x_err_bar_x = [x_lim[0] + 1, x_lim[0] + 1 + 10 * pmra_mean_err]
    x_err_bar_y = [y_lim[0] + 1 + 10 * pmdec_mean_err / 2, y_lim[0] + 1 + 10 * pmdec_mean_err / 2]
    y_err_bar_x = [x_lim[0] + 1 + 10 * pmra_mean_err / 2, x_lim[0] + 1 +  10 * pmra_mean_err / 2]
    y_err_bar_y = [y_lim[0] + 1 + 10 * pmdec_mean_err, y_lim[0] + 1]
    plt.plot(x_err_bar_x, x_err_bar_y, color = 'b')
    plt.plot(y_err_bar_x, y_err_bar_y, color = 'b')
    plt.legend(fontsize = 'xx-large', loc = 1)
    plt.show()
    
    fig, ax = plt.subplots(figsize = (10, 10))
    ax.set_aspect('equal')
    plt.scatter(list_of_points_ra, list_of_points_dec, s = 1, marker = '.', label = 'Subaru Sources')
    #plt.scatter(accepted_points['pmra'], accepted_points['pmdec'], marker = 'D')
    plt.scatter(confirmed[pmra_name], confirmed[pmdec_name], marker = '.', label = 'Confirmed Members')
    plt.scatter(in_ellipse_ra, in_ellipse_dec, marker = '*', label = 'SUBARU PM Selected Sources')
    plt.xlabel(r'$\mu_{\alpha}$' + " (mas/yr)", fontsize = 'xx-large')
    plt.ylabel(r'$\mu_{\delta}$' + " (mas/yr)", fontsize = 'xx-large')
    plt.axhline(y = 0, color = 'k', linestyle = '--', linewidth = 0.7)
    plt.axvline(x = 0, color = 'k', linestyle = '--', linewidth = 0.7)
    #plt.hlines(pm_dec - radius, pm_ra - radius, pm_ra + radius)
    #plt.hlines(pm_dec + radius, pm_ra - radius, pm_ra + radius)
    #plt.vlines(pm_ra - radius, pm_dec - radius, pm_dec + radius)
    #plt.vlines(pm_ra + radius, pm_dec - radius, pm_dec + radius)
    #plt.gca().add_artist(ellipse)
    x_lim = plt.xlim(pmra_median - 2 * pmra_std, pmra_median + 2 * pmra_std)
    y_lim = plt.ylim(pmdec_median - 2 * pmdec_std, pmdec_median + 2 * pmdec_std)
    x_err_bar_x = [x_lim[1] - 1 - 3 * pmra_mean_err, x_lim[1] - 1]
    x_err_bar_y = [y_lim[0] + 5 + 3 * pmdec_mean_err / 2, y_lim[0] + 5 + 3 * pmdec_mean_err / 2]
    y_err_bar_x = [x_lim[1] - 1 - 3 * pmra_mean_err / 2, x_lim[1] - 1 - 3 * pmra_mean_err / 2]
    y_err_bar_y = [y_lim[0] + 5 + 3 * pmdec_mean_err, y_lim[0] + 5]
    plt.plot(x_err_bar_x, x_err_bar_y, color = 'b')
    plt.plot(y_err_bar_x, y_err_bar_y, color = 'b')
    plt.legend(fontsize = 'large', loc = 3)
    plt.show()
    
    return accepted_points